In [2]:
#Shengtao Lin, Tianhe Wang

# Twitter Api

In [1]:
!pip install tweepy

In [65]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [66]:
#key and tokens
#api key: bUvI6HoYH97pwjTmnMNAPTMTS
#api secret key: Hp420vZADZ25NMR0sqEf4eVlYXVtUsyL4AhwgdAxTf2ssbr0in
#access token: 1190435999551303680-vK990ORPT4tj7KU1TTkfOtARxNzLfz
#Access Token Secret: 6KWkz1UYuCPvvGKk22AvMumRpwlA2kO8DlIVZ5KwpQtTK

access_token = "1190435999551303680-vK990ORPT4tj7KU1TTkfOtARxNzLfz"
access_token_secret = "6KWkz1UYuCPvvGKk22AvMumRpwlA2kO8DlIVZ5KwpQtTK"
consumer_key = "bUvI6HoYH97pwjTmnMNAPTMTS"
consumer_secret = "Hp420vZADZ25NMR0sqEf4eVlYXVtUsyL4AhwgdAxTf2ssbr0in"

In [67]:
import sys
default_stdout = sys.stdout
sys.stdout = open('AI-out', 'w')

In [68]:
class StdOutListener(StreamListener):
    t_count=0
    
    def on_data(self, data):
        print (data)
        self.t_count += 1
        
        #stop by 
        if self.t_count >=12000:
            sys.exit("End")
            
        return True

    def on_error(self, status):
        print (status)
    


if __name__ == '__main__':

    #This handles Twitter authetification and the connection to Twitter Streaming API
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    stream = Stream(auth, l)

    #This line filter Twitter Streams to capture data by the keywords: 'python', 'javascript', 'ruby'
    stream.filter(track=['#ArtificialIntelligence'],languages=["en"])

SystemExit: End

In [70]:
sys.stdout = default_stdout

In [30]:
import json
count=0
with open("AI-out", "r") as f1:
    for line in f1:
        count+=1
        try:
            data = json.loads(line)
            # if you want to see a specific field, you can print it. 
            #If your file is big, there may be too many of these printed
            #print(data['text']) 
        except:
            continue
print(count)

24000


# DataBase create/insert

In [20]:
#Database import
import mysql.connector
import pymongo
from datetime import datetime
import sys
import json

In [21]:
mydbsql = mysql.connector.connect(
  host="localhost",
  user="root",
  password="123456",
  database="TwitterDB"
)

In [22]:
mycursor = mydbsql.cursor(buffered=True)

In [63]:
#create DATABASE
mycursor.execute("CREATE DATABASE TwitterDB")

DatabaseError: 1007 (HY000): Can't create database 'twitterdb'; database exists

In [64]:
#Create Table
#mycursor.execute("USE TwitterDB")
mycursor.execute("CREATE TABLE IF NOT EXISTS user("+
                 "user_id VARCHAR(255) PRIMARY KEY,"+
                "name VARCHAR(255),"+
                "screen_name VARCHAR(255),"+
                "location VARCHAR(255),"+
                "description VARCHAR(255),"+
                "followers_count INT NOT NULL,"+
                "friends_count INT NOT NULL,"+
                "listed_count INT NOT NULL,"+
                "favourites_count INT NOT NULL,"+
                "statuses_count INT NOT NULL,"+
                "created_at DATETIME NOT NULL);")

In [65]:
mycursor.execute("CREATE TABLE IF NOT EXISTS post("+
                 "user_id VARCHAR(255) NOT NULL,"+
                 "post_id VARCHAR(255) NOT NULL,"+
                 "created_at DATETIME NOT NULL,"+
                 "PRIMARY KEY (user_id, post_id),"+
                 "FOREIGN KEY (user_id) REFERENCES user(user_id));")

In [58]:
#drop user table for test
sql = "DROP TABLE post"
mycursor.execute(sql)
sql = "DROP TABLE user"
mycursor.execute(sql)


In [23]:
#MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

In [24]:
#create database
mydbmongo = myclient["TwitterDB"]
#create collection
mycol = mydbmongo["Tweets"]

In [67]:
#start loading
#load function first

with open("AI-out", "r") as f1:
    for line in f1:
        
        try:
            data = json.loads(line)
            #print(data['id'])
            ProcessData(data) 
            
        except OSError as err:
            print("OS error: {0}".format(err))
        except ValueError:
            pass
        except:
            print("Unexpected error:", sys.exc_info()[0])
            raise


<ipython-input-34-58ba63afac64>:91: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  if mycol.find({'_id': tweet_id}).limit(1).count() > 0:


In [30]:
def ProcessData(data):
    
#-----------user infomation----------
    #tweet user
    if checkExist(data['user']["id"]):
        #already exist 
        updateUser(data['user'])
    else:
        #new user
        createNewUser(data['user']) 
        
        
    #retweet user
    if "retweeted_status" in data:
        if checkExist(data['retweeted_status']['user']["id"]):
            #already exist
            updateUser(data['retweeted_status']['user'])
        else:
            #new user
            createNewUser(data['retweeted_status']['user'])
            
#------------post ----------------
    newPost(data)
    
        
    

In [31]:
def checkExist(user_id):
    idquery= "SELECT COUNT(1) FROM user WHERE user_id = "+str(user_id)+";"
    mycursor.execute(idquery)
    myresult = mycursor.fetchall()

    if(myresult[0][0]!=0):
        return True
    else:
        return False
    

In [32]:
def createNewUser(data):
    user_id = data["id"]
    name = data["name"]
    screen_name=data["screen_name"]
    location = data["location"]
    description = data["description"]
    followers_count = data["followers_count"]
    friends_count = data["friends_count"]
    listed_count = data["listed_count"]
    favourites_count = data["favourites_count"]
    statuses_count = data["statuses_count"]
    
    new_datetime = datetime.strftime(datetime.strptime(data["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
    created_at = new_datetime
    
    sql = ("INSERT INTO user (user_id, name, screen_name, location,"+
    " description, followers_count, friends_count, listed_count, favourites_count, statuses_count, created_at)"+
    " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    val=(user_id,name,screen_name,location,description,int(followers_count)
         ,int(friends_count),int(listed_count),int(favourites_count),int(statuses_count),created_at)
 
    mycursor.execute(sql, val)
    mydbsql.commit()

In [33]:
def updateUser(data):
    user_id = data["id"]
    name = data["name"]
    screen_name=data["screen_name"]
    location = data["location"]
    description = data["description"]
    followers_count = data["followers_count"]
    friends_count = data["friends_count"]
    listed_count = data["listed_count"]
    favourites_count = data["favourites_count"]
    statuses_count = data["statuses_count"]
    
    sql = ("UPDATE user SET name = %s, "+
    "screen_name= %s, "+
    "location= %s, "+
    "description= %s, "+
    "followers_count= %s, "+
    "friends_count= %s, "+
    "listed_count= %s, "+
    "favourites_count= %s, "+
    "statuses_count= %s "+
    "WHERE user_id = %s")
    
    
    val = (name,screen_name,location,description,int(followers_count),
           int(friends_count),int(listed_count),int(favourites_count),int(statuses_count),user_id)
    #print(sql)
    mycursor.execute(sql, val)
    mydbsql.commit()

In [34]:
def newPost(data):
    if "retweeted_status" in data:
        # it is a RT
        RT = 1
        
        #for tweet
        if "extended_tweet" in data:
            #for tweet mongoDB
            tweet_id = data['id']
            text_full = data['extended_tweet']['full_text']
            text=text_full.partition('RT')[0]
            
            hashtag = []
            if("extended_tweet" in data['retweeted_status']):
                for ht in (data['retweeted_status']['extended_tweet']['entities']['hashtags']):
                    hashtag.append(ht['text'])
            else:
                for ht in (data['retweeted_status']['entities']['hashtags']):
                    hashtag.append(ht['text'])
        
            new_datetime = datetime.strftime(datetime.strptime(data["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
            created_at = new_datetime
        
            quote_count=data['quote_count']
            reply_count=data['reply_count']
            retweet_count=data['retweet_count']
            favorite_count=data['favorite_count']
            
            user_id=data['user']['id']
            user_screen_name=data['user']['screen_name']
            
            rt_id=data['retweeted_status']['id']
            
        
            mydict = { "_id": tweet_id,"RT":RT,"text":text,"hashtag":hashtag,"created_at":created_at,
                      "quote_count":quote_count,"reply_count":reply_count,"retweet_count":retweet_count,
                      "favorite_count":favorite_count,"user_screen_name":user_screen_name,"rt_id":rt_id}
        
            x = mycol.insert_one(mydict)
        
            #for post table mySQL
        
            sql = ("INSERT INTO post (user_id, post_id, created_at) VALUES (%s, %s, %s)")
            val=(user_id,tweet_id,created_at)
        
            mycursor.execute(sql, val)
            mydbsql.commit()
        
        else:
            #for tweet mongoDB
            tweet_id = data['id']
            text_full = data['text']
            text=text_full.partition('RT')[0]
            
            hashtag = []
            if("extended_tweet" in data['retweeted_status']):
                for ht in (data['retweeted_status']['extended_tweet']['entities']['hashtags']):
                    hashtag.append(ht['text'])
            else:
                for ht in (data['retweeted_status']['entities']['hashtags']):
                    hashtag.append(ht['text'])
        
            new_datetime = datetime.strftime(datetime.strptime(data["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
            created_at = new_datetime
        
            quote_count=data['quote_count']
            reply_count=data['reply_count']
            retweet_count=data['retweet_count']
            favorite_count=data['favorite_count']
            
            user_screen_name=data['user']['screen_name']
            rt_id=data['retweeted_status']['id']
        
            mydict = { "_id": tweet_id,"RT":RT,"text":text,"hashtag":hashtag,"created_at":created_at,
                      "quote_count":quote_count,"reply_count":reply_count,"retweet_count":retweet_count,
                      "favorite_count":favorite_count,"user_screen_name":user_screen_name,"rt_id":rt_id}
        
            x = mycol.insert_one(mydict)
        
            #for post table mySQL
            user_id=data['user']['id']
        
            sql = ("INSERT INTO post (user_id, post_id, created_at) VALUES (%s, %s, %s)")
            val=(user_id,tweet_id,created_at)
        
            mycursor.execute(sql, val)
            mydbsql.commit()
        
        #for retweet
        tweet_id = data['retweeted_status']['id']
        if mycol.find({'_id': tweet_id}).limit(1).count() > 0:
            # existed retweet
            pass
        else:
            # new retweet
            #print(data['retweeted_status'])
            insertNewPost(data['retweeted_status'])
        
    else:
        # not a RT
        insertNewPost(data)
        

In [35]:
def insertNewPost(data):
    if "extended_tweet" in data:
        #for tweet mongoDB
        tweet_id = data['id']
        RT = 0
        text = data['extended_tweet']['full_text']
        hashtag = []
        for ht in (data['extended_tweet']['entities']['hashtags']):
            hashtag.append(ht['text'])
        
        new_datetime = datetime.strftime(datetime.strptime(data["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
        created_at = new_datetime
    
        quote_count=data['quote_count']
        reply_count=data['reply_count']
        retweet_count=data['retweet_count']
        favorite_count=data['favorite_count']
            
        user_id=data['user']['id']
        user_screen_name=data['user']['screen_name']
        
        mydict = { "_id": tweet_id,"RT":RT,"text":text,"hashtag":hashtag,"created_at":created_at,
                  "quote_count":quote_count,"reply_count":reply_count,"retweet_count":retweet_count,
                  "favorite_count":favorite_count,"user_screen_name":user_screen_name}
        
        x = mycol.insert_one(mydict)
        
        #for post table mySQL
        
        sql = ("INSERT INTO post (user_id, post_id, created_at) VALUES (%s, %s, %s)")
        val=(user_id,tweet_id,created_at)
    
        mycursor.execute(sql, val)
        mydbsql.commit()
        
    else:
        #for tweet mongoDB
        tweet_id = data['id']
        RT = 0
        text = data['text']
        hashtag = []
        for ht in (data['entities']['hashtags']):
            hashtag.append(ht['text'])
        
        new_datetime = datetime.strftime(datetime.strptime(data["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
        created_at = new_datetime
        
        quote_count=data['quote_count']
        reply_count=data['reply_count']
        retweet_count=data['retweet_count']
        favorite_count=data['favorite_count']
            
        user_screen_name=data['user']['screen_name']
        
        mydict = { "_id": tweet_id,"RT":RT,"text":text,"hashtag":hashtag,"created_at":created_at,
                  "quote_count":quote_count,"reply_count":reply_count,"retweet_count":retweet_count,
                  "favorite_count":favorite_count,"user_screen_name":user_screen_name}
        
        x = mycol.insert_one(mydict)
        
        #for post table mySQL
        user_id=data['user']['id']
        
        sql = ("INSERT INTO post (user_id, post_id, created_at) VALUES (%s, %s, %s)")
        val=(user_id,tweet_id,created_at)
        
        mycursor.execute(sql, val)
        mydbsql.commit()


# Search Engine

In [18]:
import pandas as pd
import time
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
import re

<ipython-input-18-70e28d54814a>:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [50]:
def main():

In [4]:
global cache 
cache = Cache(10)

In [10]:
def processPost(posts, key):
    result=[]
    for x in posts:
        text=x['text']
        created_at=x['created_at']
        quote_count=x['quote_count']
        reply_count=x['reply_count']
        retweet_count=x['retweet_count']
        favorite_count=x['favorite_count']
        user_screen_name=x['user_screen_name']
        RT=x['RT']
        if RT==1:
            rtid=x['rt_id']
            myquery = { "_id": rtid }
            mydoc = mycol.find(myquery)
            for y in mydoc:
                rt_text=y['text']
                rt_name=y['user_screen_name']
            text = str(text)+(" @")+str(rt_name)+(" ")+str(rt_text)
        result.append([user_screen_name,text,created_at,quote_count,reply_count,retweet_count,favorite_count])
    df=pd.DataFrame(result,columns =["name","text","created_at","#quote","#reply","#retweet","#favorite"])
    
    #setting cache
    cache.put(key, df)
    
    l=len(df.index)
    i=0
    if l<=5:
        print(df)
    else:
        while i+5<l:
            print(df[i:i+5])
            i=i+5
            statement = input("Displaying 5 results with total "+str(l)+
                             " results, enter 0 to exit, enter any other key to see more.\n")
            if statement == '0':
                break
        print(df[i:l])
    
    
            

In [11]:
def searchWord(word):
    df = cache.get(word)
    if type(df) != int:
        cache.put(word, df)
        print(df)
    else:
        t0 = time.time()
        result=[]
        que=".*"+str(word)+".*"
        #regular expression search
        myquery = {"text": {"$regex" : re.compile(que)}}

        mydoc = mycol.find(myquery).sort("retweet_count", -1)
        for y in mydoc:
            result.append(y)

        t1 = time.time()
        total = t1-t0
        print("search time used: "+ str(total))    
        processPost(result, word)
    
    

In [55]:
temp=None
def searchTag(hashtag):
    t0 = time.time()
    
    df = cache.get(hashtag)
    if type(df) != int:
        cache.put(hashtag, df)
        t1 = time.time()
        total = t1-t0
        print("search time used: "+ str(total)+"\n")
        #show only 5 ouputs if df includes more than 5 
        l=len(df.index)
        i=0
        if l<=5:
            print(df)
        else:
            while i+5<l:
                print(df[i:i+5])
                i=i+5
                statement = input("Displaying 5 results with total "+str(l)+
                                 " results, enter 0 to exit, enter any other key to see more.\n")
                if statement == '0':
                    break
            print(df[i:l])

        
    
    else:
        res = []
        global temp
        myquery = {"hashtag": hashtag}
        mydoc = mycol.find(myquery).sort("retweet_count", -1)

        for y in mydoc:
            res.append(y)

        if res:
            t1 = time.time()
            total = t1-t0
            print("search time used: "+ str(total))
            temp=res
            processPost(res, hashtag)
        else:
            print("No tweets relates to this hashtag")

In [13]:
cache.get("AI").head(10)

AttributeError: 'int' object has no attribute 'head'

In [14]:
def validate(date_text):
    try:
        datetime.strptime(date_text, "%Y-%m-%d")
    except ValueError:
        raise ValueError("Incorrect data format, please use YYYY-MM-DD")
        
def searchtime(beginning, ending):
    t0 = time.time()
    res = []
    
    #validate if user input correct format of date
    validate(beginning)
    validate(ending)
    
    beginning = beginning + str(" 00:00:00")
    ending = ending + str(" 23:59:59")
    myquery = {"created_at": {
        "$gte": beginning,
        "$lte": ending
    } 
    }
    mydoc = mycol.find(myquery).sort("created_at", -1)
    
    for i in mydoc:
        res.append(i)
    
    t1 = time.time()
    total = t1-t0
    print("search time used: "+ str(total))
    processPost(res)


In [31]:
def searchUser(name):
    t0 = time.time()
    
    name_1 = name + "1"
    df = cache.get(name_1)
    if type(df) != int:
        cache.put(name_1, df)
        print(df)
    else:
        result=[]
        #get user info
        sql = "SELECT * FROM user WHERE (user_id = %s) OR (name = %s) OR (screen_name = %s);"
        val=(name,name,name)
        mycursor.execute(sql, val)
        myresult = mycursor.fetchall()
        x=myresult[0]
        df = pd.DataFrame([list(x),], columns =["id", "name", "screen_name", "location",
                                                "description","followers_count","friends_count",
                                                "listed_count","favourites_count","statuses_count","created_at"])
        cache.put(name_1, df)
        print(df)
    
    #get user posts
    name_2 = name + "2"
    df2 = cache.get(name_2)
    if type(df2) != int:
        cache.put(name_2, df2)
        print(df2)
    else:
        user_id=str(x[0])
        sql = "SELECT * FROM post WHERE (user_id= %s) ORDER BY created_at DESC;"
        val = (user_id,)
        mycursor.execute(sql, val)
        myresult = mycursor.fetchall()

        for x in myresult:
            myquery = { "_id": int(x[1]) }
            mydoc = mycol.find(myquery)
            for y in mydoc:
                result.append(y)

        t1 = time.time()
        total = t1-t0
        print("search time used: "+ str(total))
        processPost(result, name_2)
    
    

In [16]:
def search():
    while True:
        statement = input("For user search, enter 1\n"+"For tweet search, enter 2\n"+
                          "For hashtag search, enter 3\n"+"For time range search, enter 4\n"+
                          "To exit, enter 0\n")
        if(statement=='0'):
            print("Exit")
            break
        elif(statement=='1'):
            statement = input("Please enter the user name, screen name, or id:\n")
            searchUser(statement)
            break
        elif(statement=='2'):
            statement = input("Please enter the keyword:\n")
            searchWord(statement)
            break
        elif(statement=='3'):
            statement = input("Please enter the hashtag:\n")
            searchTag(statement)
            break
        elif(statement=='4'):
            statement1 = input("Please enter the begine time in format YYYY-MM-DD:\n")
            statement2 = input("Please enter the end time in format YYYY-MM-DD:\n")
            searchtime(statement1,statement2)
            break
        else:
            print("invalid input")

In [59]:
search()

For user search, enter 1
For tweet search, enter 2
For hashtag search, enter 3
For time range search, enter 4
To exit, enter 0
3
Please enter the hashtag:
AI
search time used: 0.0

             name  \
0  BenjaminP3ters   
1  AdrianZidaritz   
2  BenjaminP3ters   
3  IainLJBrown      
4  IainLJBrown      

                                                                                                                                                                                                                                                                       text  \
0  Sponsored Roundtable: Artificial Intelligence Tech - SecurityInfoWatch\n\nRead more here: https://t.co/KadJTOg31M\n\n#ArtificialIntelligence #AI #DataScience #MachineLearning #BigData #100DaysOfCode #Python #DeepLearning #NLP #Robots #IoT                             
1  This website has everything I learned about #AI in my career. 👨‍🎓 Now I can go back to grandfather duty 👨‍👧‍👧!\n\nhttps://t.co/pS3o4Xeugc\n\nPlease

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [57]:
cache.clean()

In [ ]:
if __name__ == "__main__":
    main()
    
    

In [63]:
myclient.close()

In [64]:
mycursor.close()

True

# Caching

In [6]:
from collections import deque
class Cache:
    def __init__(self, capacity=10):
        self._capacity = capacity
        self._dict = {}
        self._queue = deque([])
    
    def queue_update(self, key):
        #update queue
        if key in self._queue:
            self._queue.remove(key)
        self._queue.append(key)
    
    def get(self, key):
        self.queue_update(key)
        return self._dict.get(key, -1) # return -1 if not in cache
    
    def put(self, key, value):
        
        if key in self._dict.keys(): #if already exist
            self.queue_update(key)
        elif len(self._dict)>=self._capacity: #if not exist but over the capacity
            LRU = self._queue.popleft()
            del self._dict[LRU]
        else: #new key value pair
            self._queue.append(key)
            self._dict[key] = value
    
    def clean(self):
        self._dict.clear()
        
    def print_keys(self):
        keys = self._dict.keys()
        return keys
        

In [34]:
cache.clean()

In [35]:
cache.print_keys()

dict_keys([])